# Intermediate feature maps

In [1]:
import os
os.chdir('../')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
from model import Model

In [3]:
model = Model(num_classes=7)

In [4]:
model.summary('full')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing (Lambda)      (None, 96, 96, 3)         0         
                                                                 
 mobilenet_0.10_96 (Sequent  (None, 3, 3, 102)         39881     
 ial)                                                            
                                                                 
 pooling (GlobalMaxPooling2  (None, 102)               0         
 D)                                                              
                                                                 
 classifier (Dense)          (None, 7)                 721       
                                                                 
Total params: 40602 (158.60 KB)
Trainable params: 721 (2.82 KB)
Non-trainable params: 39881 (155.79 KB)
_________________________________________________________________


In [5]:
# Get the output size of every layer in the model in a list
base_model_maps = [layer.output_shape[1:] for layer in model.base_model.layers]
full_model_maps = [layer.output_shape[1:] for layer in model.model.layers]
intermediate_maps = base_model_maps + full_model_maps

In [6]:
sizes = []
for f_map in intermediate_maps:
    # Return the product of all dimensions of the feature map
    sizes.append(np.prod(f_map))
print(sizes)

[6912, 6912, 6912, 6912, 6912, 6912, 13824, 13824, 13824, 14406, 3456, 3456, 3456, 6912, 6912, 6912, 6912, 6912, 6912, 6912, 6912, 6912, 7500, 1728, 1728, 1728, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 4225, 900, 900, 900, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 1836, 2499, 459, 459, 459, 918, 918, 918, 918, 918, 918, 918, 918, 918, 27648, 918, 102, 7]


In [7]:
max_index = sizes.index(max(sizes))

In [8]:
print(f'Largest feature map: {sizes[max_index]} values')
size_max = sizes[max_index] * 32 / 8 / 1024
print(f'Assuming 32 bit per value: {size_max} KB')

Largest feature map: 27648 values
Assuming 32 bit per value: 108.0 KB


In [9]:
size_previous = sizes[max_index - 1] * 32 / 8 / 1024
size_following = sizes[max_index + 1] * 32 / 8 / 1024
print(f'Previous feature map: {size_previous} KB')
print(f'Following feature map: {size_following} KB')

Previous feature map: 3.5859375 KB
Following feature map: 3.5859375 KB


In [10]:
print(f'Total requirements: {size_max + max([size_previous, size_following])} KB')

Total requirements: 111.5859375 KB


In [11]:
print(f'Total requirements when quantizing the first map: {size_max / 4 + max([size_previous, size_following])} KB')

Total requirements when quantizing the first map: 30.5859375 KB


In [12]:
print(f'Total requirements when quantizing both maps: {size_max / 4 + max([size_previous, size_following]) / 4} KB')

Total requirements when quantizing both maps: 27.896484375 KB
